In [8]:
from pyspark.sql import *
import plotly.plotly as py
from pyspark.mllib.stat import Statistics
py.sign_in('harshitshah', 'rdckn6jnj9')
import numpy as np
import pandas as pd
import requests
import matplotlib
from pyspark.ml.feature import Normalizer, StandardScaler
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
#%pylab inline

#read data into pandas dataframe
df = pd.read_csv('adult.data', header=-1)
df_test = pd.read_csv('adult.test', header=-1)
df.columns = ['age','workclass','fnlwgt','education','educationNum','maritalStatus','occupation','relationship','race','sex','capitalGain','capitalLoss','hoursPerWeek','nativeCountry','income']
df_test.columns = ['age','workclass','fnlwgt','education','educationNum','maritalStatus','occupation','relationship','race','sex','capitalGain','capitalLoss','hoursPerWeek','nativeCountry','income']

def findMean(col):
    return col.mean().round()
    
def findMode(col):
    return col.mode()

#histogram for age, race and education
x0 = df['age']
#x1 = df['race']
#x2 = df['education']
#x3 = df['workclass']

trace1 = Data([Histogram(x=x0, histnorm='count', name='Age')])
#trace2 = Data([Histogram(x=x1, histnorm='count', name='Race')])
#trace3 = Data([Histogram(x=x2, histnorm='count', name='Education')])
#trace4 = Data([Histogram(x=x3, histnorm='count', name='WorkClass')])

plot_url = py.plot(trace1, filename='basic-histogram')
#plot_url2 = py.plot(trace2, filename='basic-histogram')
#plot_url3 = py.plot(trace3, filename='basic-histogram')
#plot_url4 = py.plot(trace4, filename='basic-histogram')

#Data Exploration
mode_workclass = findMode(df['workclass'])
mode_maritalStatus = findMode(df['maritalStatus'])
mode_occupation = findMode(df['occupation'])
mode_relationship = findMode(df['relationship'])
mode_race = findMode(df['race'])
mode_sex = findMode(df['sex'])
mode_nativeCountry = findMode(df['nativeCountry'])

#Data Exploration: test
mode_workclass = findMode(df_test['workclass'])
mode_maritalStatus = findMode(df_test['maritalStatus'])
mode_occupation = findMode(df_test['occupation'])
mode_relationship = findMode(df_test['relationship'])
mode_race = findMode(df_test['race'])
mode_sex = findMode(df_test['sex'])
mode_nativeCountry = findMode(df_test['nativeCountry'])

#replacing missing values with mode
df.workclass = df.workclass.replace([' ?'],[mode_workclass])
df.maritalStatus = df.maritalStatus.replace([' ?'],[mode_maritalStatus])
df.occupation = df.occupation.replace([' ?'],[mode_occupation])
df.relationship = df.relationship.replace([' ?'],[mode_relationship])
df.race = df.race.replace([' ?'],[mode_race])
df.sex = df.sex.replace([' ?'],[mode_sex])
df.nativeCountry = df.nativeCountry.replace([' ?'],[mode_nativeCountry])

#replacing missing values with mode: test
df_test.workclass = df_test.workclass.replace([' ?'],[mode_workclass])
df_test.maritalStatus = df_test.maritalStatus.replace([' ?'],[mode_maritalStatus])
df_test.occupation = df_test.occupation.replace([' ?'],[mode_occupation])
df_test.relationship = df_test.relationship.replace([' ?'],[mode_relationship])
df_test.race = df_test.race.replace([' ?'],[mode_race])
df_test.sex = df_test.sex.replace([' ?'],[mode_sex])
df_test.nativeCountry = df_test.nativeCountry.replace([' ?'],[mode_nativeCountry])

#create dummy variables
df_dummy = pd.get_dummies(df)
df_dummy.drop(df_dummy.columns[[2,6,14,30,37,51,57,62,64,105]], axis=1, inplace=True)

#create dummy variables: test
df_test_dummy = pd.get_dummies(df_test)
df_test_dummy.drop(df_test_dummy.columns[[2,6,14,30,37,51,57,62,64,105]], axis=1, inplace=True)

#normalization 
df_norm = (df_dummy - df_dummy.mean()) / (df_dummy.std())

#normalization: test 
df_test_norm = (df_test_dummy - df_test_dummy.mean()) / (df_test_dummy.std())

#write dataframe to csv
df_dummy.to_csv('df.csv',header = False)

#write dataframe to csv: test
df_test_dummy.to_csv('df_test.csv',header = False)

#defining a function to split values
def parsePoint(line):
    values = [float(x) for x in line.split(',')]
    return LabeledPoint(values[96], values[0:95])

#reading csv file into rdd
rdd = sc.textFile("df.csv")
parsedData = rdd.map(parsePoint)

#reading csv file into rdd: test
rdd_test = sc.textFile("df_test.csv")
parsedTestData = rdd_test.map(parsePoint)

#run LogisticRegressionWithLBFGS L1
model1 = LogisticRegressionWithLBFGS.train(parsedData, regType='l1')

#run LogisticRegressionWithLBFGS L2
model2 = LogisticRegressionWithLBFGS.train(parsedData, regType='l2')

labelsAndPredsL1 = parsedData.map(lambda p: (p.label, model1.predict(p.features)))
trainErrL1 = labelsAndPredsL1.filter(lambda (v, p): v != p).count() / float(parsedData.count())
testErrL1 = labelsAndPredsL1.filter(lambda (v, p): v != p).count() / float(parsedTestData.count())
print("Training Error for LogisticRegressionWithLBFGS - L1 = " + str(trainErrL1))
print("Test Error for LogisticRegressionWithLBFGS - L1 = " + str(testErrL1))

labelsAndPredsL2 = parsedData.map(lambda p: (p.label, model2.predict(p.features)))
trainErrL2 = labelsAndPredsL2.filter(lambda (v, p): v != p).count() / float(parsedData.count())
testErrL2 = labelsAndPredsL2.filter(lambda (v, p): v != p).count() / float(parsedTestData.count())
print("Training Error for LogisticRegressionWithLBFGS - L2 = " + str(trainErrL2))
print("Test Error for LogisticRegressionWithLBFGS - L2 = " + str(testErrL2))

ValueError: Length mismatch: Expected axis has 1 elements, new values have 15 elements

In [6]:
from pyspark.mllib.classification import SVMWithSGD
from pyspark.sql import *
from pyspark.mllib.stat import Statistics
import numpy as np
import pandas as pd
import requests
import matplotlib
from pyspark.ml.feature import Normalizer, StandardScaler
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
#%pylab inline

#read data into pandas dataframe
df = pd.read_csv('/Users/insignia/Desktop/Big_Data_Analytics/Midterm/adult.data.txt', header=-1)
df_test = pd.read_csv('/Users/insignia/Desktop/Big_Data_Analytics/Midterm/adult.test.txt', header=-1)
df.columns = ['age','workclass','fnlwgt','education','educationNum','maritalStatus','occupation','relationship','race','sex','capitalGain','capitalLoss','hoursPerWeek','nativeCountry','income']
df_test.columns = ['age','workclass','fnlwgt','education','educationNum','maritalStatus','occupation','relationship','race','sex','capitalGain','capitalLoss','hoursPerWeek','nativeCountry','income']

def findMean(col):
    return col.mean().round()
    
def findMode(col):
    return col.mode()

#Data Exploration
mode_workclass = findMode(df['workclass'])
mode_maritalStatus = findMode(df['maritalStatus'])
mode_occupation = findMode(df['occupation'])
mode_relationship = findMode(df['relationship'])
mode_race = findMode(df['race'])
mode_sex = findMode(df['sex'])
mode_nativeCountry = findMode(df['nativeCountry'])

#Data Exploration: test
mode_workclass = findMode(df_test['workclass'])
mode_maritalStatus = findMode(df_test['maritalStatus'])
mode_occupation = findMode(df_test['occupation'])
mode_relationship = findMode(df_test['relationship'])
mode_race = findMode(df_test['race'])
mode_sex = findMode(df_test['sex'])
mode_nativeCountry = findMode(df_test['nativeCountry'])

#replacing missing values with mode
df.workclass = df.workclass.replace([' ?'],[mode_workclass])
df.maritalStatus = df.maritalStatus.replace([' ?'],[mode_maritalStatus])
df.occupation = df.occupation.replace([' ?'],[mode_occupation])
df.relationship = df.relationship.replace([' ?'],[mode_relationship])
df.race = df.race.replace([' ?'],[mode_race])
df.sex = df.sex.replace([' ?'],[mode_sex])
df.nativeCountry = df.nativeCountry.replace([' ?'],[mode_nativeCountry])

#replacing missing values with mode: test
df_test.workclass = df_test.workclass.replace([' ?'],[mode_workclass])
df_test.maritalStatus = df_test.maritalStatus.replace([' ?'],[mode_maritalStatus])
df_test.occupation = df_test.occupation.replace([' ?'],[mode_occupation])
df_test.relationship = df_test.relationship.replace([' ?'],[mode_relationship])
df_test.race = df_test.race.replace([' ?'],[mode_race])
df_test.sex = df_test.sex.replace([' ?'],[mode_sex])
df_test.nativeCountry = df_test.nativeCountry.replace([' ?'],[mode_nativeCountry])

#create dummy variables
df_dummy = pd.get_dummies(df)
df_dummy.drop(df_dummy.columns[[2,6,14,30,37,51,57,62,64,105]], axis=1, inplace=True)

#create dummy variables: test
df_test_dummy = pd.get_dummies(df_test)
df_test_dummy.drop(df_test_dummy.columns[[2,6,14,30,37,51,57,62,64,105]], axis=1, inplace=True)

#normalization 
df_norm = (df_dummy - df_dummy.mean()) / (df_dummy.max() - df_dummy.min())

#normalization: test 
df_test_norm = (df_test_dummy - df_test_dummy.mean()) / (df_test_dummy.max() - df_test_dummy.min())

#write dataframe to csv
df_dummy.to_csv('df.csv',header = False)

#write dataframe to csv: test
df_test_dummy.to_csv('df_test.csv',header = False)

#defining a function to split values
def parsePoint(line):
    values = [float(x) for x in line.split(',')]
    return LabeledPoint(values[96], values[0:95])

#reading csv file into rdd
rdd = sc.textFile("df.csv")
parsedData = rdd.map(parsePoint)

#reading csv file into rdd: test
rdd_test = sc.textFile("df_test.csv")
parsedTestData = rdd.map(parsePoint)

#run SVMWithSGD L1
model1 = SVMWithSGD.train(parsedData, regType='l1', step=0.0001)

#run SVMWithSGD L2
model2 = SVMWithSGD.train(parsedData, regType='l2')

labelsAndPredsL1 = parsedData.map(lambda p: (p.label, model1.predict(p.features)))
labelsAndPredsL2 = parsedData.map(lambda p: (p.label, model2.predict(p.features)))

trainErrL1 = labelsAndPredsL1.filter(lambda (v, p): v != p).count() / float(parsedData.count())
testErrL1 = labelsAndPredsL1.filter(lambda (v, p): v != p).count() / float(parsedTestData.count())
print("Training Error for SVMWithSGD - L1 = " + str(trainErrL1))
print("Test Error for SVMWithSGD - L1= " + str(testErrL1))

trainErrL2 = labelsAndPredsL2.filter(lambda (v, p): v != p).count() / float(parsedData.count())
testErrL2 = labelsAndPredsL2.filter(lambda (v, p): v != p).count() / float(parsedTestData.count())
print("Training Error for SVMWithSGD - L2 = " + str(trainErrL2))
print("Test Error for SVMWithSGD - L2 = " + str(testErrL2))

CParserError: Error tokenizing data. C error: Expected 1 fields in line 2, saw 15
